# imports

In [ ]:
import pandas as pd
import numpy as np
import re

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import konlpy

# read

In [ ]:
hankyoung_sihwang = pd.read_csv('./input/hankyoung_sihwang_url.csv')
hankyoung_foreign = pd.read_csv('./input/hankyoung_foreign_url.csv')
naver_sihwang     = pd.read_csv('./input/naver_sihwang_url.csv')
naver_foreign     = pd.read_csv('./input/naver_foreign_url.csv')
infonews_sihwang  = pd.read_csv('./input/infonews_국내.csv')
infonews_foreign  = pd.read_csv('./input/infonews_해외.csv')

infonews_sihwang.dropna(inplace=True)

In [ ]:
hankyoung_sihwang.shape, hankyoung_foreign.shape, naver_sihwang.shape, naver_foreign.shape, infonews_sihwang.shape, infonews_foreign.shape, 

((63085, 4), (37345, 4), (372974, 4), (72443, 4), (43212, 4), (6826, 4))

In [ ]:
hankyoung_sihwang['class'] = '한경국내'
hankyoung_foreign['class'] = '한경해외'
naver_sihwang['class']     = '네이버국내'
naver_foreign['class']     = '네이버해외'
infonews_sihwang['class']  = '인포국내'
infonews_foreign['class']  = '인포해외'

In [ ]:
hankyoung_sihwang['date'] = hankyoung_sihwang['date'].str[:10]
hankyoung_sihwang['date'] = pd.to_datetime(hankyoung_sihwang['date'])

hankyoung_foreign['date'] = hankyoung_foreign['date'].str[:10]
hankyoung_foreign['date'] = pd.to_datetime(hankyoung_foreign['date'])

naver_sihwang['date'] = pd.to_datetime(naver_sihwang['date'], format='%Y%m%d')
naver_foreign['date'] = pd.to_datetime(naver_foreign['date'], format='%Y%m%d')

infonews_sihwang['date'] = infonews_sihwang['date'].str.split('|').str[-1].str.split(' ').str[-2]
infonews_sihwang['date'] = pd.to_datetime(infonews_sihwang['date'])

infonews_foreign['date'] = infonews_foreign['date'].str.split('|').str[-1].str.split(' ').str[-2]
infonews_foreign['date'] = pd.to_datetime(infonews_foreign['date'])

In [ ]:
df = pd.concat(
    [
        hankyoung_sihwang, hankyoung_foreign, 
        naver_sihwang, naver_foreign, 
        infonews_sihwang, infonews_foreign
    ],
)
df.reset_index(drop=True, inplace=True)

In [ ]:
df.shape

(595885, 5)

In [ ]:
df.head()

,page,date,title,url,class
0,1,2021-11-10,"코스피, 물가 부담에 하락…2930선까지 밀려",https://www.hankyung.com/finance/article/20211...,한경국내
1,1,2021-11-10,(마감)코스피 기관 2355억 매도 우위,https://www.hankyung.com/finance/article/20211...,한경국내
2,1,2021-11-10,(마감)프로그램 1707억 매도 우위,https://www.hankyung.com/finance/article/20211...,한경국내
3,1,2021-11-10,(마감)코스피 외국인 236억 순매도,https://www.hankyung.com/finance/article/20211...,한경국내
4,1,2021-11-10,"오를 땐 디커플링이었는데…코스피, 美증시 따라 하락출발",https://www.hankyung.com/finance/article/20211...,한경국내


In [ ]:
df.tail()

,page,date,title,url,class
595880,228,2014-08-27,"삼성전자, 애플에 모바일 D램 공급 재개",/news/articleView.html?idxno=4182071,인포해외
595881,228,2014-08-25,"JP모건 ""옐런 연설, 금리 중립적·균형적""",/news/articleView.html?idxno=4182070,인포해외
595882,228,2014-08-18,"JP모건 ""한은, 추가 금리인하 여지 열어둬""",/news/articleView.html?idxno=4182069,인포해외
595883,228,2014-08-14,"JP모건 ""美 2분기 GDP 전망 3.9%→3.6%""",/news/articleView.html?idxno=4182068,인포해외
595884,228,2014-08-12,"LG화학, 테슬라 맞서 신형 전기차 배터리 개발중",/news/articleView.html?idxno=4174410,인포해외


In [ ]:
df['date'].max(), df['date'].min()

(Timestamp('2021-11-10 00:00:00'), Timestamp('2000-03-17 00:00:00'))

In [ ]:
# df.to_csv('./input/articles_title.csv', index=False)

# Lemmatize

In [ ]:
import pandas as pd
import numpy as np
import re

from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import konlpy

In [ ]:
df = pd.read_csv('./input/articles_title.csv')

In [ ]:
from ckonlpy.tag import Twitter
from nltk.stem import WordNetLemmatizer

tw = Twitter()
lm = WordNetLemmatizer()

titles = df['title'].values.tolist()

headwords = []
for title in titles:
    headwords.append([lm.lemmatize(word) for word in tw.nouns(title)])

f:\workspace\venv\lib\site-packages\konlpy\tag\_okt.py:16: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [ ]:
headwords[:5]

[['코스피', '물가', '부담', '하락', '선', '까지'],
 ['마감', '코스피', '기관', '매도', '우위'],
 ['마감', '프로그램', '매도', '우위'],
 ['마감', '코스피', '외국인', '순', '매도'],
 ['땐', '디', '커플링', '었는데', '코스피', '증시', '하락', '출발']]

In [ ]:
len(headwords)

595885

# Document-Term Matrix(DTM)

In [ ]:
headwords = [' '.join(word) for word in headwords]

In [ ]:
headwords[:5]

['코스피 물가 부담 하락 선 까지',
 '마감 코스피 기관 매도 우위',
 '마감 프로그램 매도 우위',
 '마감 코스피 외국인 순 매도',
 '땐 디 커플링 었는데 코스피 증시 하락 출발']

In [ ]:
len(headwords)

595885

In [ ]:
df['headwords'] = headwords

In [ ]:
# df.to_csv('./input/articles_title_headwords.csv', index=False)